# Limpieza y preparación de datos

**Proyecto:** Análisis de comportamiento de clientes en un e-commerce

**Autor:** Sergio Naranjo Burgo

---

## Objetivo

En este _notebook_ se realiza la limpieza y preparación del conjunto de datos de ventas de un e-commerce.

El objetivo es dejar los datos listos para el análisis exploratorio y posteriores modelos de análisis o predicción.

Se incluyen pasos de:

- Inspección inicial de los datos
- Eliminación de duplicados y valores nulos
- Filtrado de cantidades y precios inválidos
- Conversión de fechas
- Guardado del conjunto de datos limpio

In [14]:
# Importamos las librerías necesarias

import pandas as pd
import numpy as np

# Definir rutas de archivos
ruta_raw = "../data/raw/OnlineRetail.csv"            # CSV original
ruta_processed = "../data/processed/online_retail_clean.csv"   # CSV limpio

## 1.- Carga de datos

Se carga el conjunto de datos original desde 'data/raw/OnlineRetail.csv'.

In [8]:
df = pd.read_csv(ruta_raw, encoding='latin1')

# Mostrar las primeras filas
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2.- Exploración inicial

Se inspeccionan las primeras filas, tipos de datos y estadísticas generales para identificar problemas con valores nulos, duplicados o errores.

In [9]:
# Información general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [10]:
# Estadísticas descriptivas
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


### Observaciones de la exploración inicial

- 'InvoiceNo': número de factura, puede contener letras (por cancelaciones).
- 'StockCode': código del producto.
- 'Description': nombre del producto, tiene algunos valores nulos.
- 'Quantity': hay valores negativos o extremadamente altos, que representan devoluciones o errores.
- 'InvoiceDate': actualmente es texto, necesita convertirse a _datetime_.
- 'UnitPrice': hay valores negativos o extremadamente altos.
- 'CustomerID': hay filas nulas que no podemos usar para análisis de clientes.
- 'Country': país del cliente, sin problemas. 

## 3.- Limpieza de datos

Se aplican los siguientes pasos:

1. Eliminación de filas sin 'CustomerID'.
2. Eliminación de duplicados exactos.
3. Filtrado de cantidades <=0 (devoluciones o errores).
4. Filtrado de precios <=0 (errores o promociones negativas).
5. Conversión de 'InvoiceDate' a _datetime_.
6. Rellenar valores nulos de 'Description' con "Sin descripción".
7. Opcional: filtrar _outliers_ extremos para mejorar visualizaciones.

In [12]:
# 1. Eliminar filas sin 'CustomerID'
df = df.dropna(subset=['CustomerID'])

# 2. Eliminar duplicados exactos
df = df.drop_duplicates()

# 3. Filtrar cantidades y precios inválidos
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

# 4. Convertir fechas a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=False)

# 5. Rellenar descripciones nulas
df['Description'] = df['Description'].fillna("Sin descrpción")

# 6. Opcional: filtrar outliers extremos
df = df[df['Quantity'] < 1000]
df = df[df['UnitPrice'] < 1000]

# 7. Verificación final
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 392557 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392557 non-null  object        
 1   StockCode    392557 non-null  object        
 2   Description  392557 non-null  object        
 3   Quantity     392557 non-null  int64         
 4   InvoiceDate  392557 non-null  datetime64[ns]
 5   UnitPrice    392557 non-null  float64       
 6   CustomerID   392557 non-null  float64       
 7   Country      392557 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.0+ MB


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Comentarios sobre la limpieza

- Se eliminaron filas sin 'CustomerID' para poder analizar clientes de manera precisa.
- Se elimninaron duplicados exactos para evitar conteos dobles.
- Cantidades y precios inválidos se filtraron para mantener datos representativos de ventas reales.
- Fechas convertidas a _datetime_ permiten análisis temporal posterior.
- Descripciones nulas rellenadas para no perder información de productos.
- _Outliers_ extremos eliminados para mejorar futuras visualizaciones.

## 4.- Guardado del conjunto de datos limpio

Los datos limpios se guardan en 'data/processed/' para poder ser utilizados en el análisis exploratorio, visualizaciones o moddelos predictivos posteriores.

In [13]:
# Guardar CSV limpio
df.to_csv(ruta_processed, index=False)

## Resumen del Notebook 01

- Conjunto de datos original: 541.909 filas, con valores nuylos y errores.
- Conjunto de datos limpio: menos filas, sin valores nulos y listo para análisis.
- Próximo paso: Notebook 02 - Análisis exploratorio de datos (EDA), donde se realizarán gráficos de ventas, clientes y productos. 